In [ ]:
# =============================================================================
# Author: Deepthi Kuttichiara and Pubudu Sanjeewani Thihagoda Gamage
# Date: 20-08-2024
# Title: A Novel Non-Iterative Training Method for CNN Classifiers Using Gram-Schmidt Process
# License: MIT License
# =============================================================================

# Import necessary libraries for model loading, image processing, evaluation, and data handling
import numpy as np
from tensorflow.keras.models import load_model 
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
import scipy.io
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

# Set the index of the layer from which features will be extracted
l=5 
class_num = 10

# Load MNIST dataset (handwritten digit images)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape grayscale images to have a single channel
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# One-hot encode class labels for training and testing
y_train = to_categorical(y_train, class_num)
y_test = to_categorical(y_test, class_num)

# Define a function to resize images to 32x32 and convert to 3-channel RGB format
def change_size(image):
    img = array_to_img(image, scale=False) # Convert array to PIL Image
    img = img.resize((32, 32))             # Resize image
    img = img.convert(mode='RGB')          # Convert to RGB
    arr = img_to_array(img)                # Convert back to array
    return arr.astype(np.float64)

# Resize and convert all training and test images
x_train = [change_size(img) for img in x_train]
x_train = np.array(x_train)
x_train.shape

x_test = [change_size(img) for img in x_test]
x_test = np.array(x_test)
x_test.shape

# Normalize pixel values to [0, 1]
x_train= (x_train.astype('float32'))/255
x_test= (x_test.astype('float32'))/255

# Load the pre-trained VGG16-based model for MNIST
model=load_model("mnist_vgg16.h5")

# Print all layer indices, names, and output shapes for inspection
for i in range(len(model.layers)):
	layer = model.layers[i]
	print(i, layer.name, layer.output.shape)

# Create a new model to extract features from the specified intermediate layer
new_model = Model(inputs=model.inputs, outputs=model.layers[l].output)

# Generate feature representations for test and train datasets
test_features=new_model.predict(x_test)
train_features=new_model.predict(x_train)

# Save extracted features to .npy files
np.save(r'C:\Users\sanje\Desktop\Griffith\code\pretrained-MNIST\mnisttest_2000relu2.npy',test_features)
np.save(r'C:\Users\sanje\Desktop\Griffith\code\pretrained-MNIST\mnisttrain_2000relu2.npy',train_features)

# Analyze the sparsity (number of non-zero elements) in the extracted features
l=np.sum(train_features,axis=0)
c=np.count_nonzero(l) 

0 vgg16 (None, 1, 1, 512)
1 flatten (None, 512)
2 dense (None, 4000)
3 dense_1 (None, 2000)
4 dropout (None, 2000)
5 dense_2 (None, 1000)
6 dropout_1 (None, 1000)
7 dense_3 (None, 500)
8 dropout_2 (None, 500)
9 dense_4 (None, 10)
313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 136ms/step
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 299s 159ms/step
